# NBA Teams, Payroll and Performance
## Ashley Kang, Min-Yi Chen, Chloe Yueh
## Overview
## Motivation
## Data Sources:
- https://www.kaggle.com/datasets/loganlauton/nba-players-and-team-data?select=NBA+Player+Box+Score+Stats%281950+-+2022%29.csv
- https://www.kaggle.com/datasets/loganlauton/nba-players-and-team-data?select=NBA+Player+Stats%281950+-+2022%29.csv
- https://www.kaggle.com/datasets/loganlauton/nba-players-and-team-data?select=NBA+Salaries%281990-2023%29.csv
